### The Culture of International Relations

#### About this project
Cultural treaties are the bi-lateral and multilateral agreements among states that promote and regulate cooperation and exchange in the fields of life generally call cultural or intellectual. Although it was only invented in the early twentieth century, this treaty type came to be the fourth most common bilateral treaty in the period 1900-1980 (Poast et al., 2010). In this project, we seek to use several (mostly European) states’ cultural treaties as a historical source with which to explore the emergence of a global concept of culture in the twentieth century. Specifically, the project will investigate the hypothesis that the culture concept, in contrast to earlier ideas of civilization, played a key role in the consolidation of the post-World War II international order.

The central questions that interest me here can be divided into two groups: 
- First, what is the story of the cultural treaty, as a specific tool of international relations, in the twentieth century? What was the historical curve of cultural treaty-making? For example, in which political or ideological constellations do we find (the most) use of cultural treaties? Among which countries, in which historical periods? What networks of relations were thereby created, reinforced, or challenged? 
- Second, what is the "culture" addressed in these treaties? That is, what do the two signatories seem to mean by "culture" in these documents, and what does that tell us about the role that concept played in the international system? How can quantitative work on this dataset advance research questions about the history of concepts?

In this notebook, we deal with these treaties in three ways:
1) quantitative analysis of "metadata" about all bilateral cultural treaties signed betweeen 1919 and 1972, as found in the World Treaty Index or WTI (Poast et al., 2010).
    For more on how exactly we define a "cultural treaty" here, and on other principles of selection, see... [add this, using text now in "WTI quality assurance"].
2) network analysis of the system of international relationships created by these treaties (using data from WTI, as above).
3) Text analysis of the complete texts of selected treaties. 

After some set-up sections, the discussion of the material begins at "Part 1," below.

### <span style='color:blue'>**Mandatory Prepare Step**</span>: Setup Notebook
Setup HTML appearance.

In [14]:
%%html
<style>
.widget-vbox .widget-label {width: 60px; }
.widget-toggle-button { width: 100px; }
.jupyter-widgets { font-size: 8pt; }
.widget-label { font-size: 8pt;}
.widget-vbox .widget-label { width: 60px; }
.widget-dropdown select { font-size: 8pt; width: 102px; }
.widget-select > select { font-size: 8pt; }
.widget-toggle-buttons .widget-toggle-button { font-size: 8pt; }
.widget-inline-hbox .widget-readout {
    min-width: 50px;
    font-size: 8pt;
    margin-left: 2px;
    text-align: left;
    vertical-align: top;}
.widget-readout.overflow { box-shadow: none; }
.widget-slider, .widget-hslider { width: 200px; height: 18px; }
button[title*='7CORR'] { margin-left: 71px; }
</style>

In [15]:
# Setup
%load_ext autoreload
%autoreload 2

import os
import warnings
import ipywidgets as widgets
import pandas as pd

from bokeh.plotting import output_notebook
from bokeh.palettes import Set1_8 as line_palette
from bokeh.io import push_notebook

os.sys.path = os.sys.path if '..' in os.sys.path else os.sys.path + ['..']

warnings.filterwarnings('ignore')

import common.config

from common.color_utility import StaticColorMap
from common.file_utility import FileUtility
from common.widgets_utility import WidgetUtility
from common.utility import extend, flatten, getLogger, SimpleStruct
from common.treaty_state import load_wti_index, default_period_specification
from common.network.layout import layout_network
from common.network.networkx_utility import create_nx_subgraph, get_positioned_edges2, get_positioned_nodes

from network_analysis_gui import display_network_analyis_gui
from network_analysis_plot import plot_network, get_palette
from network_analysis import create_party_network, slice_network_datasource, setup_node_size, adjust_node_label_offset

logger = getLogger(name='cultural_treaties')

wti_index = load_wti_index('../data')

#unknown_parties = wti_index.check_party()
#logger.warning('[{} UNKNOWN PARTIES] '.format(len(unknown_parties)) + (' '.join([ '"' + str(x) + '"' for x in unknown_parties])))

output_notebook()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2018-10-09 13:24:53,516 : INFO : Data loaded!


Loading BokehJS ...

### Network Visualization of Signed Treaties
        
- **TODO: Split result into one graph per category**

In [16]:
# Visualize treaties

plot_data = SimpleStruct(handle=None, nodes=None, edges=None, slice_range_type=2, slice_range=(1900, 2000))

from network_analysis_gui import NETWORK_PLOT_OPTS

def display_partial_party_network(slice_range_type, slice_range, slice_changed_callback):
    global plot_data
    nodes, edges = slice_network_datasource(plot_data.edges, plot_data.nodes, slice_range, slice_range_type)

    plot_data.edges_source.data.update(edges)
    plot_data.nodes_source.data.update(nodes)
    
    plot_data.slice_range_type = slice_range_type
    plot_data.slice_range = slice_range
    
    sign_dates = plot_data.edges_source.data['signed']
    
    if len(sign_dates) == 0:
        return
    
    min_year, max_year = min(sign_dates).year, max(sign_dates).year

    slice_changed_callback(min_year, max_year)

    print('Slice range by {}: {}-{}, edge count: {} ({})'.format(
        'id' if slice_range_type == 1 else 'Year',
        slice_range[0], slice_range[1],
        len(plot_data.edges_source.data['signed']), len(plot_data.edges['signed'])))
    
    push_notebook(handle=plot_data.handle)
    
party_data = None
def display_party_network(
    parties,
    period_group,
    treaty_filter,
    # plot_data,
    topic_group=None,
    recode_is_cultural=False,
    layout_algorithm='',
    C=1.0,
    K=0.10,
    p1=0.10,
    output='network',
    party_name='party',
    node_size_range=[40,60],
    refresh=False,
    palette_name=None,
    width=900,
    height=900,
    node_size=None,
    node_partition=None,
    wti_index=None,
    progress_callback=None,
    done_callback=None
    ):
    try:
        global plot_data, party_data
        progress = progress_callback
    
        weight_threshold = 0.0

        palette = get_palette(palette_name)
        
        progress(1)
        
        kwargs = dict(
            period_group=period_group,
            treaty_filter=treaty_filter,
            recode_is_cultural=recode_is_cultural
        )
        parties=list(parties)
        party_data = wti_index.get_party_network(party_name, topic_group, parties, **kwargs)
        
        if party_data is None or party_data.shape[0] == 0:
            print('No data for selection')
            return
        
        if topic_group is not None:

            party_data = party_data.loc[(party_data.topic.isin(topic_group.keys()))]
            
            group_keys = topic_group.keys()
            line_palette_map = { k: i % len(line_palette) for i, k in enumerate(group_keys) }
            party_data['line_color'] = party_data.category.apply(lambda x: line_palette[line_palette_map[x]])

        else:
            party_data['category'] = party_data.topic
            
        party_data['edge_label'] = party_data.signed.apply(lambda x: x.year).astype(str) + '/' + party_data.category

        progress(2)
        
        #if not multigraph:
        #    data = data.groupby(['party', 'party_other']).size().reset_index().rename(columns={0: 'weight'})
        
        G = create_party_network(party_data, K, node_partition, palette) #, multigraph)
        
        progress(3)

        if output == 'network':

            if weight_threshold > 0:
                G = get_sub_network(G, weight_threshold)
            
            layout, _ = layout_network(G, layout_algorithm, **dict(scale=1.0, K=K, C=C, p=p1))

            progress(4)

            edges = get_positioned_edges2(G, layout, sort_attr='signed')
            nodes = get_positioned_nodes(G, layout)
            
            edges = { k: list(edges[k]) for k in edges }
            nodes = { k: list(nodes[k]) for k in nodes }
    
            node_size = setup_node_size(nodes, node_size, node_size_range)
            
            x_offset, y_offset = adjust_node_label_offset(nodes, node_size)
            
            plot_opts = extend(NETWORK_PLOT_OPTS,
                               figsize=(width, height),
                               node_size=node_size,
                               node_label_opts=dict(y_offset=y_offset, x_offset=x_offset),
                               edge_label_opts={}
                        )
            
            progress(5)
            
            data = plot_network(nodes=nodes, edges=edges, node_label='name', edge_label='edge_label', **plot_opts)
 
            plot_data.update(**data)
            
            progress(6)
            
            #bp.show(p)
            
            if done_callback is not None:
                done_callback(None)
                
        elif output == 'table':
            party_data.columns = [ dict(party='source', party_other='target').get(x,x) for x in party_data.columns ]
            display(party_data)
            
    except Exception as ex:
        print(ex)
        raise
    finally:
        progress(0)
    

display_network_analyis_gui(wti_index, display_party_network, display_partial_party_network, plot_data)
